#다층 퍼셉트론으로 손글씨 숫자 분류[실습]

###import

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

# 항상 같은 결과를 갖기 위해 랜덤 시드 설정
tf.random.set_seed(678)

#라이브러리
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from IPython.display import Image

###load dataset

In [ ]:
#텐서플로의 MSIST 데이터셋 불러오기
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape)
print(x_test.shape)
# 데이터 셋은 6만개의 학습 데이터, 1만개의 테스트 데이터
# 가로 28, 세로 28픽셀

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)


###dataset check

In [ ]:
#픽셀 범위는 0-255, 0은 흰색, 255는 검은색
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

#0-255까지 그레이 스케일 확인
print(x_train[0][8])
print(y_train[0:9])

train data has 60000 samples
every train data is 28 * 28 image
[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]
[5 0 4 1 9 2 1 3 1]


###normalization

In [ ]:
#데이터를 255로 나눔으로써 모든 값을 0,1로 정규화
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

###model summary

In [ ]:
#다층 퍼셉트론의 입력 레이어에 데이터 넣기위해, 2차원 -> 1차원 형태로 데이터 형태를 바꾼다
#행렬 데이터 -> 배열 데이터
model = Sequential([
    Flatten(input_shape=(28, 28)), # 데이터 차원 변경
    Dense(256, activation='relu'), # 첫번째 히든 레이어 (h1)
    Dense(128, activation='relu'), # 두번째 히든 레이어 (h2)
    Dropout(0.1), # 두번째 히든 레이어(h2)에 드랍아웃(10%) 적용
    Dense(10), # 세번째 히든 레이어 (logit)
    Activation('softmax') # softmax layer
])

model.summary()
#flatten & softmax는 노드가 없으므로 파라미터 존재 안함
#첫번째 레이어가 784개 입력 받는 256개 노드 존재, 노드마다 편향값 하나씩 있으므로
#784*256+256=200960개의 파라미터 존재

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 256)               200960    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
Total params: 235,146
Trainable params: 235,146
Non-trai

###model compile

In [ ]:
#손실함수, 최적화 방법 모델에 적용
"""
sparse_categorical_crossentropy:
레이블을 원 핫 인코딩으로 자동으로 변경하여 크로스 엔트로피 측정합니다.
"""
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

###model train start

In [ ]:
#매 주기마다(Epoch) 검증 데이터로 검증 정확도 측정, 5번 연속으로 개선되지 않는다면
#조기 종료, 이때 검증 정확도가 가장 높은 모델 저장
callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=False),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

model.fit(x_train, y_train, epochs=300, batch_size=1000, validation_split = 0.1, callbacks=callbacks)

Epoch 1/300
54/54 [==============================] - 2s 28ms/step - loss: 0.6740 - accuracy: 0.8117 - val_loss: 0.2151 - val_accuracy: 0.9403
Epoch 2/300
54/54 [==============================] - 1s 25ms/step - loss: 0.2391 - accuracy: 0.9307 - val_loss: 0.1459 - val_accuracy: 0.9638
Epoch 3/300
54/54 [==============================] - 1s 25ms/step - loss: 0.1713 - accuracy: 0.9505 - val_loss: 0.1238 - val_accuracy: 0.9665
Epoch 4/300
54/54 [==============================] - 1s 25ms/step - loss: 0.1341 - accuracy: 0.9607 - val_loss: 0.1037 - val_accuracy: 0.9687
Epoch 5/300
54/54 [==============================] - 1s 25ms/step - loss: 0.1091 - accuracy: 0.9683 - val_loss: 0.0911 - val_accuracy: 0.9730
Epoch 6/300
54/54 [==============================] - 1s 25ms/step - loss: 0.0922 - accuracy: 0.9729 - val_loss: 0.0866 - val_accuracy: 0.9742
Epoch 7/300
54/54 [==============================] - 1s 25ms/step - loss: 0.0770 - accuracy: 0.9778 - val_loss: 0.0807 - val_accuracy: 0.9763
Epoch 

### result

In [ ]:
results = model.evaluate(x_test,  y_test, verbose = 0)
print('test loss, test acc:', results)

test loss, test acc: [0.07280179113149643, 0.9789999723434448]
